In [3]:
import modules.singleton_reduction as singleton
import modules.isolate_reduction as isolate
from tqdm import tqdm
import importlib
importlib.reload(singleton)
importlib.reload(isolate)
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import io
import copy


In [4]:
# Load the full graph
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

In [5]:
importlib.reload(singleton)
importlib.reload(isolate)

G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
# isolatedNodes = list(nx.isolates(G))
G, isolatedNum = isolate.isolate_reduction(G)

Node count before singleton reduction:  58742
Nodes before:  52124
Nodes after:  19257


In [6]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
init_infected = [9]
max_steps = 100
G_temp = copy.deepcopy(G)

# Uncomment below block if you want to only view nodes that are accessible from init_infected
G_temp = G_temp.subgraph(sir.get_accessible_sus_nodes(G_temp, init_infected)).copy()

In [7]:
# Uncomment below block if you want to only view nodes that are accessible from init_infected
# print("Nodes to calculate: ", len(G_temp))
# pos = nx.spring_layout(G_temp, seed=8020, gravity=0.75)

In [8]:
importlib.reload(sir)

_, _, _, _, infotext_total, constants_total = sir.sir_model(G_temp, G_full_sr, init_infected=init_infected, infection_rate=0.2, recovery_rate=0.1, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0.0001,0.01), network_type='full')

[DEBUG2] False
Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  1
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG

In [9]:
infotext_total

[{'ssw': 950,
  'sso': 5668,
  'it': 1,
  'st': 4990,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 4164, 'i': 120724},
  'afterloss': {'q': 4164, 'i': 88272},
  'i_prevented': 0,
  'ss': 6618,
  'is': 0,
  'rs': 0},
 {'ssw': 817,
  'sso': 5668,
  'it': 587,
  'st': 4404,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 4164, 'i': 120724},
  'afterloss': {'q': 4164, 'i': 88272},
  'i_prevented': 1876,
  'ss': 6485,
  'is': 133,
  'rs': 0},
 {'ssw': 694,
  'sso': 5668,
  'it': 1124,
  'st': 3867,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 4164, 'i': 120724},
  'afterloss': {'q': 4164, 'i': 88272},
  'i_prevented': 3931,
  'ss': 6362,
  'is': 256,
  'rs': 0},
 {'ssw': 581,
  'sso': 5668,
  'it': 1561,
  'st': 3430,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 4164, 'i': 120724},
  'afterloss': {'q': 4164, 'i': 88272},
  'i_prevented': 6339,
  'ss': 6249,
  'is': 369,
  'rs': 0},
 {'ssw': 496,
  'sso': 5668,
  'it': 1954,
  'st': 3037,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 4164, 'i': 120724},
  'afterloss': {'q': 4164,

In [10]:
constants_total

{'virusFoundStep': 4, 'tn': 58742, 'xt': 53751}

In [11]:
# Graphical Analysis of a single run
plt.close()

# create data
x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

loss_total = [i['loss']['q']+i['loss']['i'] for i in infotext_total]
max_loss_total = max(loss_total)
loss_total = [i/max_loss_total*(constants_total['tn']-constants_total['xt']) for i in loss_total]


# plot lines
plt.plot(x, st_total, label = "Total susceptible", alpha=0.5, color='g')
plt.plot(x, it_total, label = "Total infected", alpha=0.5, color='r')
plt.plot(x, rt_total, label = "Total recovered", alpha=0.5, color='b')
plt.plot(x, qt_total, label = "Total quarantined", alpha=0.5, color='0.7')
plt.plot(x, loss_total, label = "Loss (scaled)", alpha=0.5, color='k')
plt.vlines(constants_total['virusFoundStep'], 0, constants_total['tn']-constants_total['xt'], color='k', linestyle='dotted', lw=1)
plt.legend()
plt.savefig(f"results/test.png", format="PNG", bbox_inches='tight')

In [12]:
# Loss analysis of a single run
plt.close()
loss = {'quarantine': 1, 'infected': 1}

x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

plt.stackplot(x, (
                    [i*loss['infected'] for i in it_total],
                    [i*loss['quarantine'] for i in qt_total]
                ),
             labels=['Loss from infected', 'Loss from quarantines'],
             colors=['#ff0000','#444444'],
              alpha=0.8)
plt.legend()
plt.savefig(f"results/test2.png", format="PNG", bbox_inches='tight')

In [14]:
importlib.reload(sir)
import os
from multiprocessing import Pool
from functools import partial
quarantine_lengths = [1, 5, 10, 15, 20]
values = []
for i in quarantine_lengths:
    # Loss analysis of multiple runs
    run_count = 1000

    init_infected = [9]
    # init_infected = 
    max_steps = 1000
    infection_rate = 0.2
    recovery_rate = 0.1
    noticeability_rates = (0.01,0.1)
    quarantine_length = i
    contact_tracing = 3
    loss_matrix = ((1000, 0.5),(0.2, 0.1))

    loss_results = None

    tempList = []
    for i in range(run_count):
        tempList.append(G)

    worker = partial(
        sir.simulate_threaded,
        G=G, 
        G_full_sr=G_full_sr, 
        init_infected=init_infected, 
        infection_rate=infection_rate, 
        recovery_rate=recovery_rate, 
        max_steps=max_steps, 
        noticeability_rates=noticeability_rates, 
        quarantine_length=quarantine_length,
        contact_tracing=contact_tracing,
        loss_matrix=loss_matrix,
        tempList=tempList,
    )

    with Pool(19) as p:
        results = p.map(worker, range(run_count))

    loss_results = [x['loss'] for x in results]
    afterloss_results = [x['afterloss'] for x in results]
    recovered_results = [x['rt'] for x in results]

    total_loss_results = [x['q'] + x['i'] for x in loss_results] 
    total_afterloss_results = [x['q'] + x['i'] for x in afterloss_results] 

    print(f"Total Loss: {np.mean(total_loss_results)} -- Standard Dev.: {np.std(total_loss_results)}")
    print(f"Infected Loss: {np.mean([x['i'] for x in loss_results])} -- Infected After-Loss: {np.mean([x['i'] for x in loss_results])}")
    print(f"Total After-loess: {np.mean(total_afterloss_results)} -- Standard Dev.: {np.std(total_afterloss_results)}")
    print(f"Infected After-Loss: {np.mean([x['i'] for x in afterloss_results])} -- Quarantined After-Loss: {np.mean([x['q'] for x in afterloss_results])}")
    print(f"Average total infected nodes: {np.mean(recovered_results)}")
    values.append(("Total Loss: ", np.mean(total_loss_results), "Total Loss - Standard Deviation: ", np.std(total_loss_results), "Infected Loss: ", np.mean([x['i'] for x in loss_results]), "Infected After-Loss: ", np.mean([x['i'] for x in loss_results]), "Total After-loss: ", np.mean(total_afterloss_results), "Total After-loss - Standard Deviation: ", np.std(total_afterloss_results), "Infected after-loss: ", np.mean([x['i'] for x in afterloss_results]), "Quarantined after-loss: ", np.mean([x['q'] for x in afterloss_results]), "Average infected nodes: ", np.mean(recovered_results)))
print(values)

Run:  1
Run:  2
Run:  14
Run:  3
Run:  15
Run:  4
Run:  16
Run:  28
Run:  5
Run:  17
Run:  29
Run:  18
Run:  30
Run:  19
Run:  42
Run:  31
Run:  6
Run:  43
Run:  32
Run:  7
Run:  44
Run:  33
Run:  8
Run: Run:   4520

Run:  56
Run:  9
Run:  46
Run:  57
Run:  21
Run:  10
Run:  47
Run:  58
Run:  22
Run:  34
Run:  11
Run:  70
Run:  59
Run:  23
Run:  35
Run:  12
Run:  71
Run:  60
Run:  36
Run: Run:   24
13
Run:  72
Run:  61
Run:  48
Run:  37
Run:  73
Run:  25
Run: Run:   3849

Run:  74
Run:  84
Run:  26
Run:  85
Run:  39
Run:  75
Run:  27
Run:  51
Run:  40
Run:  86
Run:  62
Run:  87
Run:  52
Run:  41
Run:  63
Run:  98
Run:  88
Run:  53
Run:  64
Run:  76
Run:  99
Run:  54
Run:  89
Run:  65
Run:  77
Run:  55
Run:  78
Run:  101
Run:  66
Run:  112
Run:  102
Run:  79
Run:  67
Run:  113
Run:  90
Run:  80
Run:  103
Run:  68
Run:  114
Run:  126Run: 
 91
Run:  69
Run:  81
Run:  115
Run:  127
Run:  92
Run:  82
Run:  116
Run:  128
Run:  93
Run:  83
Run:  104
Run:  117
Run:  129
Run:  94
Run:  140
Run:

In [15]:
j = 0 
for i in values:
    print(f"Quarantine length: ", quarantine_lengths[j]) 
    print(i[0], i[1])
    print(i[2], i[3])
    print(i[4], i[5])
    print(i[6], i[7])
    print(i[8], i[9])
    print(i[10], i[11])
    print(i[12], i[13])
    print(i[14], i[15])
    print(i[16], i[17])

    print("\n")
    j += 1

Quarantine length:  1
Total Loss:  3042665.1427000007
Total Loss - Standard Deviation:  635304.2893308619
Infected Loss:  3038552.622600001
Infected After-Loss:  3038552.622600001
Total After-loss:  1952370.2235000003
Total After-loss - Standard Deviation:  636730.5260283188
Infected after-loss:  1948257.7033999998
Quarantined after-loss:  4112.5201
Average infected nodes:  3033.433


Quarantine length:  5
Total Loss:  2825062.670100001
Total Loss - Standard Deviation:  772019.6442375901
Infected Loss:  2820593.6266000015
Infected After-Loss:  2820593.6266000015
Total After-loss:  1724246.0021000006
Total After-loss - Standard Deviation:  763904.240915034
Infected after-loss:  1719776.9586000005
Quarantined after-loss:  4469.043500000002
Average infected nodes:  2815.915


Quarantine length:  10
Total Loss:  2659783.604100001
Total Loss - Standard Deviation:  736714.8706366058
Infected Loss:  2654861.784000001
Infected After-Loss:  2654861.784000001
Total After-loss:  1569409.042099999

In [2]:
print(f"Infected Loss: {np.mean([x['i'] for x in loss_results])} -- Infected After-Loss: {np.mean([x['i'] for x in loss_results])}")


NameError: name 'np' is not defined

In [193]:
plt.cla()
plt.hist(total_loss_results, bins=200)
plt.savefig("results/test3.png")

# loss_matrix = ((1, 0),(0.2, 0.1)) w/ 100 runs
## max_steps = 100
quarantine_length 5: 15342.928
quarantine_length 5, ct level 2: 14599.878000000002
quarantine_length 10: 15261.373
quarantine_length 5, ct level 1, nt_rate * 10 noticed: 14576.289000000004
quarantine_length inf, ct level 1: 20178.598

## max_steps = 1000 (*)
quarantine_length 5, ct level 1: 14514.140000000009
quarantine_length 10, ct level 1: 15413.901000000007
quarantine_length 5, ct level 2: 14708.250000000007
quarantine_length 5, ct level 3: 15418.860000000004
quarantine_length 2, ct level 1: 15211.508000000003
no quarantines: 14869.428000000004

# loss_matrix = ((1, 0),(0.2, 0.1)) w/ 1000 runs
## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.2 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01)
no quarantining: ~14470 loss and ~3637 std
quarantine_length 5, ct level 0: ~14492 loss and ~3610 std
quarantine_length 10, ct level 0: ~14804 loss and ~3727 std
quarantine_length 5, ct level 1: ~14948 loss and ~3747 std
quarantine_length 10, ct level 1: ~15314 loss and ~3720 std
quarantine_length 5, ct level 2: ~14725 loss and ~3844 std
quarantine_length 10, ct level 2: ~15281 loss and ~4066 std
quarantine_length 5, ct level 3: ~14989 loss and ~3863 std
quarantine_length 10, ct level 3: ~15204 loss and ~3892 std

## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.1 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01) 
no quarantining: ~12231 loss and ~3892 std
quarantine_length 5, ct level 0: ~12197 loss and ~3997 std
quarantine_length 10, ct level 0: ~12284 loss and ~3977 std
quarantine_length 5, ct level 1: ~12591 loss and ~4007 std (q: 437 loss -- i: 11927)
quarantine_length 10, ct level 1: ~12502 loss and ~3913 std (q: 706 loss -- i: 11796)
quarantine_length 5, ct level 2: ~12287 loss and ~3809 std (q: 411 loss -- i: 11876)
quarantine_length 10, ct level 2: ~12673 loss and ~4074 std (q: 725 loss -- i: 11948)

# loss_matrix = ((1000, 0.5),(0.2, 0.1)) w/ 1000 runs
## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.2 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01)
no quarantining: ~3976276 loss and ~524603 std
quarantine_length 5, ct level 0: ~3942146 loss and ~547844 std (q: 350 -- i: 3941796)
quarantine_length 10, ct level 0: ~3966693 loss and ~531257 std (q: 490 -- i: 3966203)
quarantine_length 5, ct level 1: ~3920625 loss and ~538986 std (q: 986 -- i: 3919639)
quarantine_length 10, ct level 1: ~3918447 loss and ~530026 std (q: 1320 -- i: 3917127)
quarantine_length 5, ct level 2: ~3881514 loss and ~542880 std (q: 947 -- i: 3880568)
quarantine_length 10, ct level 2: ~3932767 loss and ~533736 std (q: 1298 -- i: 3931469)
quarantine_length 5, ct level 3: ~3973245 loss and ~502952 std (q: 938 -- i: 3972307)

quarantine_length 10, ct level 3: ~3942986 loss and ~531066 std (q: 949 -- i: 3942037)




7.865,665400001
